In [4]:
# !pip install --upgrade pip
# !pip install torch==1.13.1
# !pip install torchdata==0.5.1
# !pip install transformers==4.27.2
# !pip install datasets==2.11.0

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

##1. Summarize Dialogue without Prompt Engineering

In [3]:
huggingface_dataset_name="knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [5]:
example_indices=[40,200,1]

dash_line='-'.join('' for x in range(100))

for i,index in enumerate(example_indices):
  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print('Input Dialogue :')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('Summary :')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Dialogue :
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Summary :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  2
-----

In [6]:
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [8]:
sentence='What time is it, Niranjan?'

sentence_encoded=tokenizer(sentence,return_tensors='pt')

sentence_decoded=tokenizer.decode(sentence_encoded['input_ids'][0],skip_special_tokens=True)

print("input sentences :",sentence)
print("encoded sentences :",sentence_encoded)
print("decoded sentences :",sentence_decoded)

input sentences : What time is it, Niranjan?
encoded sentences : {'input_ids': tensor([[ 363,   97,   19,   34,    6, 2504, 2002, 7066,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
decoded sentences : What time is it, Niranjan?


In [9]:
for i, index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  inputs=tokenizer(dialogue,return_tensors='pt')

  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_length=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print('Input Dialogue :')
  print(dialogue)
  print(dash_line)
  print('Summary :')
  print(summary)
  print(dash_line)
  print("model generated summary:")
  print(output)
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Dialogue :
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Summary :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
model generated summary:
Person1: It's ten to nine.
-----------------------------------------------------------------

##2. Summarize Dialogue with an Instruction Prompt

###2.1 Zero-shot inference

In [10]:
for i, index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""

  Summarize the following conversations.
  {dialogue}

  Summary:
  """
  inputs=tokenizer(prompt,return_tensors='pt')

  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_length=50,
      )[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print('Input prompt :')
  print(prompt)
  print(dash_line)
  print('Human Summary :')
  print(summary)
  print(dash_line)
  print("model generated summary:")
  print(output)
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input prompt :


  Summarize the following conversations.
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

  Summary:
  
---------------------------------------------------------------------------------------------------
Human Summary :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
model generated summary:
The train is about to leave.

##2.2 One-shot Learning

In [16]:
def make_prompt(example_indices_full, example_indices_to_summarize):
  prompt=''

  for index in example_indices_full:
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']

    prompt+=f"""

    Dialogue:
    {dialogue}

    what was going on?
    {summary}

    """

  dialogue=dataset['test'][example_indices_to_summarize]['dialogue']

  prompt+=f"""

  Dailogur.
  {dialogue}

  what was going on?
  """

  return prompt

In [17]:
example_indices_full=[10]
example_indices_to_summarize=1

prompt=make_prompt(example_indices_full, example_indices_to_summarize)

print(prompt)



    Dialogue:
    #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

    what was going on?
    #Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

    

  Dailogur.
  #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this af

In [18]:
summary=dataset['test'][example_indices_to_summarize]['summary']

inputs=tokenizer(prompt,return_tensors='pt')

output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_length=50,
    )[0],
    skip_special_tokens=True
)


print(dash_line)
print('Input prompt :')
print(prompt)
print(dash_line)
print('Human Summary :')
print(summary)
print(dash_line)
print("one-shot model generated summary:")
print(output)
print(dash_line)
print()

---------------------------------------------------------------------------------------------------
Input prompt :


    Dialogue:
    #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

    what was going on?
    #Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

    

  Dailogur.
  #Person1#: Ms. Dawson, I need you to take a dictation for

##2.3 Few-shot Learning

In [19]:
example_indices_full=[10,100,200,50,2]
example_indices_to_summarize=1

prompt=make_prompt(example_indices_full, example_indices_to_summarize)

print(prompt)



    Dialogue:
    #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

    what was going on?
    #Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

    

    Dialogue:
    #Person1#: OK, that's a cut! Let's start from the beginning, everyone.
#Person2#: What was the problem that time?
#Person1#: The feeling was all wrong, Mike. She is tel

In [20]:
summary=dataset['test'][example_indices_to_summarize]['summary']

inputs=tokenizer(prompt,return_tensors='pt')

output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_length=50,
    )[0],
    skip_special_tokens=True
)


print(dash_line)
print('Input prompt :')
print(prompt)
print(dash_line)
print('Human Summary :')
print(summary)
print(dash_line)
print("one-shot model generated summary:")
print(output)
print(dash_line)
print()

---------------------------------------------------------------------------------------------------
Input prompt :


    Dialogue:
    #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

    what was going on?
    #Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

    

    Dialogue:
    #Person1#: OK, that's a cut! Let's start from the beg